# AIDM7330 Basic Programming for Data Science

## Pandas Examples

In [1]:
import pandas as pd
import numpy as np

**Example 1** Read the 'movie_metadata.csv' file. Tell:

    a) how many tuples and fields the dataset has.
    b) what are the fields

In [2]:
from google.colab import drive
drivePath = '/content/drive' #please do not change
drive.mount(drivePath)

Mounted at /content/drive


In [3]:
# Install the library on your environment
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=97a4824c03cff106890e27e2ba28cc5ace6195f14d23b6b5cd043b093c023a3f
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [4]:
# Import the library
import wget

# Setup URL and path variables
baseURL = 'https://raw.githubusercontent.com/pmengoni/AIDM7330-2223S1/main/'
doc = 'movie_metadata.csv'
fullURL = baseURL + doc

dataPath = drivePath + '/MyDrive/Colab Notebooks/data'

# Download the file
fileName = wget.download(fullURL, out=dataPath)

# Print the file name including the local path
print(fileName)

/content/drive/MyDrive/Colab Notebooks/data/movie_metadata.csv


In [5]:
movieDf = pd.read_csv(fileName)

movieShape = movieDf.shape
numTuples = movieShape[0]
numFields = movieShape[1]#number of column
print('{} tuples and {} fields \n'.format(numTuples, numFields))

movieColName = list(movieDf.columns)
print(movieColName)

5041 tuples and 11 fields 

['director_name', 'num_critic_for_reviews', 'duration', 'gross', 'genres', 'movie_title', 'num_voted_users', 'num_user_for_reviews', 'country', 'budget', 'imdb_score']


**Example 2** Please go through the following process to handle missing values
    
    a) Find out the sum of missing values for each column
    b) Drop all tuples containing any missing value
    
Tell how many tuples are dropped. Explain whether it is a good way to handle missing value for the case?
(please print out your answer)

In [6]:
# return a copy of cleaned df
cleanMovieDf = movieDf.dropna(how='any')#any missing data,a tuple contain any missing value it should be delete;if"all",make sure all value are missing can delete

# sum up NAs
print(movieDf.isna().sum())#check which row is na and sum them

numDropped = numTuples - cleanMovieDf.shape[0]

print('\n{} tuples dropped.'.format(numDropped))
print('This is not a good way to handle missing values since too much information gets lost.')

director_name             104
num_critic_for_reviews     50
duration                   15
gross                     884
genres                      0
movie_title                 0
num_voted_users             0
num_user_for_reviews       21
country                     5
budget                    492
imdb_score                  0
dtype: int64

1154 tuples dropped.
This is not a good way to handle missing values since too much information gets lost.


**Example 3** Try the following methods to handle missing values inplace.

    a) replace missing values in 'director_name'with 'unknown'
    b) use mean to fill in missing values in'gross' and 'budget'
    c) use median to fill in missing values in 'num_critic_for_reviews'and 'num_user_for_reviews'
    d) use mode to fill in missing values in 'country'
    e) replace missing values in 'duration' with the last observation
    
Recall from math:
* The "mean" is the "average": you add up all the numbers and then divide by the number of numbers.
* The "median" is the "middle" value in the list of numbers.
* The "mode" is the value that occurs most often. If no number in the list is repeated, then there is no mode for the list.

In [7]:
# replace with unknown
movieDf.fillna({'director_name': 'unknown'}, inplace=True)

# replace with mean
meanBudget = movieDf['budget'].mean()
meanGross = movieDf['gross'].mean()

movieDf.fillna({'budget': meanBudget,
                 'gross': meanGross}, inplace=True)

# replace with median
medianCritic = movieDf['num_critic_for_reviews'].median()
medianUser = movieDf['num_user_for_reviews'].median()

movieDf.fillna({'num_critic_for_reviews': medianCritic,
                 'num_user_for_reviews': medianUser
                }, inplace=True)

# replace with mode
modeCountry = movieDf['country'].mode()#mode:the most frequent value in dataset
movieDf.fillna({'country': modeCountry[0]}, inplace=True)

# replace with the last observation
movieDf['duration'].fillna(method='ffill', inplace=True)

movieDf.head()

,director_name,num_critic_for_reviews,duration,gross,genres,movie_title,num_voted_users,num_user_for_reviews,country,budget,imdb_score
0,James Cameron,723.0,178.0,7.605058e+08,Action|Adventure|Fantasy|Sci-Fi,Avatar,886204,3054.0,USA,2.370000e+08,7.9
1,Gore Verbinski,302.0,169.0,3.094042e+08,Action|Adventure|Fantasy,Pirates of the Caribbean: At World's End,471220,1238.0,USA,3.000000e+08,7.1
2,Sam Mendes,602.0,148.0,2.000742e+08,Action|Adventure|Thriller,Spectre,275868,994.0,UK,2.450000e+08,6.8
3,Christopher Nolan,813.0,164.0,4.481306e+08,Action|Thriller,The Dark Knight Rises,1144337,2701.0,USA,2.500000e+08,8.5
4,Doug Walker,110.0,164.0,4.846077e+07,Documentary,Star Wars: Episode VII - The Force Awakens ...,8,156.0,USA,3.976350e+07,7.1


**Example 4** There are some duplicated rows in the dataset.

    a) tell how many duplicated rows are in the dataset
    b) remove duplicates
    c) Tell how many rows left after removing duplicates

In [8]:
# number of duplications
duplicateDf = movieDf[movieDf.duplicated()]
print('{} duplicated rows'.format(duplicateDf.shape[0]))

# remove duplicates
movieDf.drop_duplicates(inplace=True)

# how many left?
print('{} rows left after removing duplicates'.format(movieDf.shape[0]))

47 duplicated rows
4994 rows left after removing duplicates


**Example 5** Create a new column named 'imdb_label'. The lable is given based on the following criteria:(important)

    a) imdb_score in (0, 4], label 'bad'
    b) imdb_score in (4, 7], label 'moderate'
    c) imdb_score in (7, 10], label 'good'
You should consider **data binning** to finish the task.

In [9]:
# define lables and categories
labels = ['bad', 'moderate', 'good']#define your labels
category = [0., 4., 7., 10.]#0-4,4-7,7-10

# create the news column
movieDf['imdb_label'] = pd.cut(movieDf['imdb_score'], labels=labels, bins=category, include_lowest=False)#panda organizethe'imdb_label' in new column,sql cannot do it

movieDf.head()

,director_name,num_critic_for_reviews,duration,gross,genres,movie_title,num_voted_users,num_user_for_reviews,country,budget,imdb_score,imdb_label
0,James Cameron,723.0,178.0,7.605058e+08,Action|Adventure|Fantasy|Sci-Fi,Avatar,886204,3054.0,USA,2.370000e+08,7.9,good
1,Gore Verbinski,302.0,169.0,3.094042e+08,Action|Adventure|Fantasy,Pirates of the Caribbean: At World's End,471220,1238.0,USA,3.000000e+08,7.1,good
2,Sam Mendes,602.0,148.0,2.000742e+08,Action|Adventure|Thriller,Spectre,275868,994.0,UK,2.450000e+08,6.8,moderate
3,Christopher Nolan,813.0,164.0,4.481306e+08,Action|Thriller,The Dark Knight Rises,1144337,2701.0,USA,2.500000e+08,8.5,good
4,Doug Walker,110.0,164.0,4.846077e+07,Documentary,Star Wars: Episode VII - The Force Awakens ...,8,156.0,USA,3.976350e+07,7.1,good


**Example 6** Try to get the following info from the data:

    a) basic statistical characteristics of each numerical feature
    b) basic statistical characteristics of each non-numerical feature
    c) the distributions(normalized) of imdb_label

In [10]:
# numerical summary
movieDf.describe()

,num_critic_for_reviews,duration,gross,num_voted_users,num_user_for_reviews,budget,imdb_score
count,4994.000000,4994.000000,4.994000e+03,4.994000e+03,4994.000000,4.994000e+03,4994.000000
mean,139.595314,107.201442,4.835009e+07,8.348511e+04,271.369844,3.976266e+07,6.441229
std,120.924777,25.253307,6.174430e+07,1.381180e+05,376.677070,1.967030e+08,1.124022
min,1.000000,7.000000,1.620000e+02,5.000000e+00,1.000000,2.180000e+02,1.600000
25%,50.000000,93.000000,8.382841e+06,8.560000e+03,65.000000,7.000000e+06,5.800000
50%,110.000000,103.000000,3.730253e+07,3.430050e+04,156.000000,2.300000e+07,6.600000
75%,193.000000,118.000000,5.137692e+07,9.612075e+04,323.000000,4.000000e+07,7.200000
max,813.000000,511.000000,7.605058e+08,1.689764e+06,5060.000000,1.221550e+10,9.500000


In [11]:
# non-numerical summary
movieDf.describe(include=['object', 'category'])

,director_name,genres,movie_title,country,imdb_label
count,4994,4994,4994,4994,4994
unique,2398,914,4915,64,3
top,unknown,Drama,Home,USA,moderate
freq,103,235,3,3777,3246


In [12]:
# distribution
movieDf['imdb_label'].value_counts(normalize = True)#if without normalize,it the exact count

moderate    0.649980
good        0.315579
bad         0.034441
Name: imdb_label, dtype: float64

**Example 7** Sort the dataset in following two ways:

    a) by budget in ascending order
    b) by budget and imdb_score in descending order
    

In [13]:
# budget in ascending order
movieDf.sort_values(by='budget').head()

,director_name,num_critic_for_reviews,duration,gross,genres,movie_title,num_voted_users,num_user_for_reviews,country,budget,imdb_score,imdb_label
4797,Jonathan Caouette,72.0,88.0,5.920140e+05,Biography|Documentary,Tarnation,5709,114.0,USA,218.0,7.2,good
5040,Jon Gunn,43.0,90.0,8.522200e+04,Documentary,My Date with Drew,4285,84.0,USA,1100.0,6.6,moderate
5038,Benjamin Roberds,13.0,76.0,4.846077e+07,Drama|Horror|Thriller,A Plague So Pleasant,38,3.0,USA,1400.0,6.3,moderate
5034,Anthony Vallone,110.0,84.0,4.846077e+07,Crime|Drama,The Mongol King,36,1.0,USA,3250.0,7.8,good
5024,Olivier Assayas,81.0,110.0,1.360070e+05,Drama|Music|Romance,Clean,3924,39.0,France,4500.0,6.9,moderate


In [14]:
# budget and imdb_score in descending order
movieDf.sort_values(by=['budget', 'imdb_score'], ascending=[False, False]).head()#means descending

,director_name,num_critic_for_reviews,duration,gross,genres,movie_title,num_voted_users,num_user_for_reviews,country,budget,imdb_score,imdb_label
2986,Joon-ho Bong,363.0,110.0,2201412.0,Comedy|Drama|Horror|Sci-Fi,The Host,68883,279.0,South Korea,1.221550e+10,7.0,moderate
3857,Chan-wook Park,202.0,112.0,211667.0,Crime|Drama,Lady Vengeance,53508,131.0,South Korea,4.200000e+09,7.7,good
3003,Lajos Koltai,73.0,134.0,195888.0,Drama|Romance|War,Fateless,5603,45.0,Hungary,2.500000e+09,7.1,good
2323,Hayao Miyazaki,174.0,134.0,2298191.0,Adventure|Animation|Fantasy,Princess Mononoke,221552,570.0,Japan,2.400000e+09,8.4,good
2334,Katsuhiro Ôtomo,105.0,103.0,410388.0,Action|Adventure|Animation|Family|Sci-Fi|Thriller,Steamboy,13727,79.0,Japan,2.127520e+09,6.9,moderate


**Example 8** Practice the following steps:

    a) Group the dataset by 'imdb_label'
    b) Select 'gross', 'budget' and 'imdb_score' to display
    c) Apply aggregation functions to the selected columns, including mean, std, min, max
    

In [15]:
# define columns to show
columnsToShow = ['gross', 'budget', 'imdb_score']

# performing grouping and apply functions to groups
movieDf.groupby(['imdb_label'])[columnsToShow].agg([np.mean, np.std, np.min, np.max])#select only 3 column to show,apply mea,std,min and max to each column

gross                                           budget  \
                    mean           std    amin         amax          mean   
imdb_label                                                                  
bad         3.065475e+07  2.198250e+07  5561.0  107285004.0  2.108896e+07   
moderate    4.332774e+07  5.010190e+07   162.0  652177271.0  4.022903e+07   
good        6.062556e+07  8.139130e+07   728.0  760505847.0  4.084008e+07   

                                                imdb_score                      
                     std     amin          amax       mean       std amin amax  
imdb_label                                                                      
bad         2.518674e+07  17350.0  1.600000e+08   3.311047  0.573024  1.6  4.0  
moderate    2.215569e+08   1100.0  1.221550e+10   6.042606  0.710380  4.1  7.0  
good        1.463408e+08    218.0  4.200000e+09   7.603871  0.424029  7.1  9.5

**Example 9** Split the 'imdb_label' column into three numerical fields:

    a)'label_bad'
    b)'label_morderate'
    c)'label_good'.
Use 1 to indicated the correct label and 0 inversely. Write this news dataframe to 'movie_metadata_new.csv'

In [16]:
# convert categorical variables into dummy/indicator variables
dfDummies = pd.get_dummies(movieDf['imdb_label'], prefix='label')#we apply dummies to every label,good for machine learning

# concatenate with the original dataframe
encodedMovieDf = pd.concat([movieDf, dfDummies], axis=1)

# write to new csv file
encodedMovieDf.to_csv(dataPath + '/' + 'movie_metadata_new.csv')

encodedMovieDf.head()

,director_name,num_critic_for_reviews,duration,gross,genres,movie_title,num_voted_users,num_user_for_reviews,country,budget,imdb_score,imdb_label,label_bad,label_moderate,label_good
0,James Cameron,723.0,178.0,7.605058e+08,Action|Adventure|Fantasy|Sci-Fi,Avatar,886204,3054.0,USA,2.370000e+08,7.9,good,0,0,1
1,Gore Verbinski,302.0,169.0,3.094042e+08,Action|Adventure|Fantasy,Pirates of the Caribbean: At World's End,471220,1238.0,USA,3.000000e+08,7.1,good,0,0,1
2,Sam Mendes,602.0,148.0,2.000742e+08,Action|Adventure|Thriller,Spectre,275868,994.0,UK,2.450000e+08,6.8,moderate,0,1,0
3,Christopher Nolan,813.0,164.0,4.481306e+08,Action|Thriller,The Dark Knight Rises,1144337,2701.0,USA,2.500000e+08,8.5,good,0,0,1
4,Doug Walker,110.0,164.0,4.846077e+07,Documentary,Star Wars: Episode VII - The Force Awakens ...,8,156.0,USA,3.976350e+07,7.1,good,0,0,1
